In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# update import path
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'src'))

import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from importlib import reload
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.neural_network import MLPRegressor

# no warnings on dataframe slices
pd.options.mode.chained_assignment = None

In [ ]:
### read dataframe from data cleaning assignment ###

# start training

In [ ]:
# select features and target
y = data1['boat_speed']
x = data1[['aoa_abs', 'wind_speed']]

# split test/train sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

### train and fit a linear model ###

# show performance
pred_y = model.predict(x_test)
print ('MAE: ', mean_absolute_error(pred_y, y_test))

In [ ]:
### normalize data before linear model using pipeline ###

In [ ]:
### use polynomial features ###

In [ ]:
### try gradient boosting ###

In [ ]:
### try some other sklearn models ###

In [ ]:
# function to predict boat speeds for all angles
def predict_boat_speeds(model, wind_speed):
    aoa = np.linspace(0, 180, 181)
    df = pd.DataFrame()
    df['aoa'] = aoa
    df['wind_speed'] = wind_speed
    df['pred'] = model.predict(df[['aoa', 'wind_speed']])
    df['vmg'] = df['pred'] * np.cos(np.radians((df['aoa'])))
    return df

# predict speeds for 20 knots
df = predict_boat_speeds(model, 20)

# show predicted speed and vmg
plt.plot(df['aoa'], df['pred'])
plt.plot(df['aoa'], df['vmg'])
plt.legend()

In [ ]:
# show polar plot
def show_polar(model):
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, polar=True)

    for wind_speed in [5, 10, 15, 20, 25]:
        df = predict_boat_speeds(model, wind_speed)
        _ = ax.plot(np.radians(df['aoa']), df['pred'], label=wind_speed)

    ax.set_thetamin(0)
    ax.set_thetamax(180)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    plt.legend()
    plt.show()

show_polar(model)

In [ ]:
# predict tack and gybe angles for all wind speeds
data = pd.DataFrame()
for wind_speed in range(2, 50):
    df = predict_boat_speeds(model, wind_speed)
    row = {
        'wind_speed': wind_speed,
        'upwind_twa': df['vmg'].idxmax(),
        'downwind_twa': df['vmg'].idxmin(),
    }
    data = data.append([row])

# wind speed as index
data = data.set_index('wind_speed')

# save to file
data.to_pickle(os.path.join('..', 'src', 'strategies', 'my_strategy', 'angles.pkl'))

# show rows
data